# Video Nonsense:

## Initial Conditions:

In [ ]:
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

In [ ]:
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

In [ ]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))

## One Class for Showing Video:

In [ ]:
class VideoShow:
    """
    Class that continuously shows a frame using a dedicated thread.
    """
    def __init__(self, ret, frame, subtitle):
        
        self.grab = ret
        self.video_frame = frame
        self.subtitle = subtitle
        self.video_stream = cv2.VideoCapture(0)
        
        (self.grabbed, self.frame) = self.video_stream.read()
        self._frame = queue.Queue()
        self._ret = queue.Queue()
        self.stopped = False

    def __enter__(self):
#         Thread(target=self.show, args=()).start()
        thread = threading.Thread(target=self._do)
        return self
    
    def _do(self):
        while not self.stopped:
            if not self.grabbed:
                self.stop()
            else:
                (self.grabbed, self.frame) = self.video_stream.read()
                self._frame.put(self.frame)
                self._ret.put(self.grabbed)
    
    def show(self):
        while not self.stopped:
            video_frame = self._frame.get()
            self.grab = self._ret.get()
            
            if (self.grab):
                outframe = hdmi_out.newframe()
                outframe[:] = self.video_frame
                output_frame = subtitle_output(self.subtitle, outframe)
                hdmi_out.writeframe(output_frame)
            else:
                raise RuntimeError("Error while reading from camera.")
                
    def __exit__(self, type, value, traceback):
        self.stopped = True

In [ ]:
ret, frame = videoIn.read()
subtitle = 'Testing 1, 2, 3'
i = 0
# video_player = VideoShow(frame, subtitle, videoIn)
with VideoShow(ret, frame, subtitle) as video_player:
    if ret:
        vid = video_player.__enter__()
        video_getter = video_player.show()
        i += 1
        if video_player.stopped:
            video_player.__exit__()

#         frame = video_player.get()
#         video_shower.frame = frame
        
i

## Two Classes:

In [ ]:
class VideoGet:
    """
    Class that continuously gets frames from a VideoCapture object
    with a dedicated thread.
    """

    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False

    def start(self):    
        thread = threading.Thread(target=self.get, args=()).start()
        return self

    def get(self):
        while not self.stopped:
            if not self.grabbed:
                self.stop()
            else:
                (self.grabbed, self.frame) = self.stream.read()
                return self.grabbed, self.frame

    def stop(self):
        self.stopped = True

In [ ]:
class VideoShow:
    """
    Class that continuously shows a frame using a dedicated thread.
    """

    def __init__(self, frame):
        self.frame = frame
        self.stopped = False

    def start(self):
        thread = threading.Thread(target=self.show, args=()).start()
        return self

    def show(self):
        while not self.stopped:
            outframe = hdmi_out.newframe()
            outframe[:] = self.frame
            hdmi_out.writeframe(outframe)
            if KeyboardInterrupt:
                self.stopped = True

    def stop(self):
        self.stopped = True


In [ ]:
def threadBoth(source=0):
    """
    Dedicated thread for grabbing video frames with VideoGet object.
    Dedicated thread for showing video frames with VideoShow object.
    Main thread serves only to pass frames between VideoGet and
    VideoShow objects/threads.
    """

    video_getter = VideoGet(source).start()
    video_shower = VideoShow(video_getter.frame).start()

    while True:
        if video_getter.stopped or video_shower.stopped:
            video_shower.stop()
            video_getter.stop()
            break

        frame = video_getter.frame
        video_shower.frame = frame

In [ ]:
threadBoth(0)

In [ ]:
videoIn.release()
hdmi_out.stop()
del hdmi_out